# 🎓 ADVANCED UNIVERSITY ADMISSION PREDICTION SYSTEM
## Hệ thống Dự báo Điểm chuẩn Đại học Y Dược 2026

---

### 🚀 Hệ thống này làm gì?

**2 Mục tiêu chính:**

1. **📊 Dự đoán PERCENTILE điểm chuẩn 2026**
   - Sử dụng: Time Series Models (WA, ETS, ARIMA, LR)
   - Ensemble Learning + Cross-Validation
   - Output: "Ngành Y khoa sẽ lấy Top 0.5% năm 2026"

2. **🎯 Dự đoán XÁC SUẤT ĐẬU của thí sinh**
   - Sử dụng: Classification Models (Random Forest, Gradient Boosting)
   - **KHÔNG dùng `gap` (điểm - điểm_chuẩn)** vì chưa biết điểm chuẩn 2026!
   - Output: "Với 27 điểm B00, bạn có 65% cơ hội đậu Y khoa"

---

### 💡 Tại sao có v1, v2, v3?

| Version | Cách tiếp cận | Vấn đề |
|---------|---------------|--------|
| v1, v2 | Dùng `gap = điểm - điểm_chuẩn` | ❌ **Data leakage!** Năm 2026 chưa có điểm chuẩn |
| **v3** | Chỉ dùng `percentile` + pattern lịch sử | ✅ **Realistic!** |

---

**Author**: AI University Admission Predictor V2.0  
**Data**: Điểm chuẩn Y Dược 2018-2025 + Điểm thi THPT 2018-2025

## 📦 1. Setup & Installation

In [ ]:
# Cài đặt thư viện cần thiết (chạy 1 lần)
# !pip install statsmodels plotly kaleido scikit-learn pandas numpy matplotlib seaborn joblib

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import glob
import os
import re
import joblib
import warnings
from datetime import datetime
from collections import defaultdict

# ML & Stats
from statsmodels.tsa.holtwinters import ExponentialSmoothing
from statsmodels.tsa.arima.model import ARIMA
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import mean_squared_error, mean_absolute_error, accuracy_score, roc_auc_score

warnings.filterwarnings("ignore")
plt.style.use('seaborn-v0_8-whitegrid')

print("✅ Import thành công!")
print(f"📅 {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

## 📤 2. Load Data (Local - VS Code)

In [ ]:
# Config paths - Điều chỉnh nếu cần
SCORE_FOLDER = 'diem_thi_thptqg'
BENCHMARK_FILE = 'diem_chuan_cleaned.csv'
OUTPUT_DIR = 'model_artifacts'

# Tạo thư mục output nếu chưa có
os.makedirs(OUTPUT_DIR, exist_ok=True)

print(f"📁 Score folder: {SCORE_FOLDER}")
print(f"📁 Benchmark file: {BENCHMARK_FILE}")
print(f"📁 Output dir: {OUTPUT_DIR}")

In [ ]:
# Kiểm tra files có tồn tại không
print("📋 Kiểm tra files...")

# Check benchmark
if os.path.exists(BENCHMARK_FILE):
    print(f"  ✅ {BENCHMARK_FILE}")
else:
    print(f"  ❌ {BENCHMARK_FILE} - KHÔNG TÌM THẤY!")

# Check score files
score_files = glob.glob(os.path.join(SCORE_FOLDER, "*.csv"))
print(f"\n📊 Tìm thấy {len(score_files)} file điểm thi:")
for f in sorted(score_files):
    print(f"  ✅ {os.path.basename(f)}")

In [ ]:
# Load data
df_benchmark = pd.read_csv('diem_chuan_cleaned.csv')

print("="*50)
print("📊 THÔNG TIN DỮ LIỆU")
print("="*50)
print(f"Số dòng: {len(df_benchmark):,}")
print(f"Số cột: {len(df_benchmark.columns)}")
print(f"Các năm: {sorted(df_benchmark['nam'].unique())}")
print(f"Số trường: {df_benchmark['university_id'].nunique()}")
print(f"Số ngành: {df_benchmark['ma_nganh'].nunique()}")

df_benchmark.head()

## 📊 3. Exploratory Data Analysis (EDA)

In [ ]:
# 3.1 Overview Statistics
print("📈 THỐNG KÊ TỔNG QUAN")
print("="*50)

yearly_stats = df_benchmark.groupby('nam').agg({
    'diem_chuan': ['mean', 'std', 'min', 'max', 'count'],
    'university_id': 'nunique',
    'ma_nganh': 'nunique'
}).round(2)

yearly_stats.columns = ['Điểm TB', 'Std', 'Min', 'Max', 'Số dòng', 'Số trường', 'Số ngành']
yearly_stats

In [ ]:
# 3.2 Interactive Box Plot - Điểm theo năm
fig = px.box(df_benchmark, x='nam', y='diem_chuan',
             color='nam',
             title='📊 Phân bố Điểm chuẩn theo Năm (2018-2025)',
             labels={'nam': 'Năm', 'diem_chuan': 'Điểm chuẩn'},
             color_discrete_sequence=px.colors.sequential.Viridis)

fig.update_layout(
    height=500,
    showlegend=False,
    xaxis_title="Năm",
    yaxis_title="Điểm chuẩn"
)
fig.show()

In [ ]:
# 3.3 Top 10 trường điểm cao nhất 2025
df_2025 = df_benchmark[df_benchmark['nam'] == 2025]
top_schools = df_2025.groupby('ten_truong')['diem_chuan'].max().sort_values(ascending=True).tail(10)

fig = go.Figure(go.Bar(
    x=top_schools.values,
    y=top_schools.index,
    orientation='h',
    marker=dict(color=top_schools.values, colorscale='Reds')
))

fig.update_layout(
    title='🏆 Top 10 Trường có Điểm chuẩn cao nhất (2025)',
    xaxis_title='Điểm chuẩn',
    yaxis_title='',
    height=500
)
fig.show()

In [ ]:
# 3.4 Xu hướng điểm chuẩn theo thời gian - Top 5 ngành
top_nganh = df_benchmark.groupby('ten_nganh').size().sort_values(ascending=False).head(5).index.tolist()
df_top = df_benchmark[df_benchmark['ten_nganh'].isin(top_nganh)]
trend = df_top.groupby(['nam', 'ten_nganh'])['diem_chuan'].mean().reset_index()

fig = px.line(trend, x='nam', y='diem_chuan', color='ten_nganh',
              title='📈 Xu hướng Điểm chuẩn Top 5 Ngành (2018-2025)',
              labels={'nam': 'Năm', 'diem_chuan': 'Điểm chuẩn TB', 'ten_nganh': 'Ngành'},
              markers=True)

fig.update_layout(height=500, legend=dict(orientation="h", yanchor="bottom", y=-0.3))
fig.show()

In [ ]:
# 3.5 Heatmap - Điểm chuẩn theo Trường x Năm
pivot = df_benchmark.pivot_table(values='diem_chuan', index='ten_truong', columns='nam', aggfunc='max')
pivot = pivot.loc[pivot.max(axis=1).sort_values(ascending=False).head(12).index]

fig = px.imshow(pivot,
                title='🔥 Heatmap Điểm chuẩn Max theo Trường x Năm',
                labels=dict(x="Năm", y="Trường", color="Điểm"),
                color_continuous_scale='RdYlGn',
                aspect='auto')

fig.update_layout(height=500)
fig.show()

In [ ]:
# 3.6 Violin Plot - Phân bố điểm theo tổ hợp
top_blocks = df_benchmark['to_hop_mon'].value_counts().head(8).index.tolist()
df_blocks = df_benchmark[df_benchmark['to_hop_mon'].isin(top_blocks)]

fig = px.violin(df_blocks, x='to_hop_mon', y='diem_chuan', color='to_hop_mon',
                title='🎻 Phân bố Điểm chuẩn theo Tổ hợp môn',
                labels={'to_hop_mon': 'Tổ hợp', 'diem_chuan': 'Điểm chuẩn'},
                box=True, points='outliers')

fig.update_layout(height=500, showlegend=False)
fig.show()

## ⚙️ 4. Configuration & Block Map

In [ ]:
# Tổ hợp môn Y Dược
BLOCK_MAP = {
    'A00': ['toan', 'vat_ly', 'hoa_hoc'],
    'A01': ['toan', 'vat_ly', 'ngoai_ngu'],
    'A02': ['toan', 'vat_ly', 'sinh_hoc'],
    'B00': ['toan', 'hoa_hoc', 'sinh_hoc'],
    'B08': ['toan', 'sinh_hoc', 'ngoai_ngu'],
    'D01': ['toan', 'ngu_van', 'ngoai_ngu'],
    'D07': ['toan', 'hoa_hoc', 'ngoai_ngu'],
    'D08': ['toan', 'sinh_hoc', 'ngoai_ngu'],
    'D13': ['toan', 'ngu_van', 'sinh_hoc'],
}

print("📚 TỔ HỢP MÔN Y DƯỢC")
print("="*50)
for block, subjects in BLOCK_MAP.items():
    subject_names = [s.replace('_', ' ').title() for s in subjects]
    print(f"  {block}: {' + '.join(subject_names)}")

## 📊 5. Build Percentile Lookup Table

In [ ]:
def build_percentile_lookup(score_folder):
    """Xây dựng bảng tra cứu percentile từ điểm thi THPT"""
    print("\n" + "="*60)
    print("📊 XÂY DỰNG PHÂN PHỐI ĐIỂM THI")
    print("="*60)

    lookup_dict = {}
    year_stats = []
    files_list = glob.glob(os.path.join(score_folder, "*.csv"))

    if not files_list:
        print(f"❌ Không tìm thấy file trong {score_folder}")
        return {}, []

    print(f"📁 Tìm thấy {len(files_list)} file")

    for file_path in sorted(files_list):
        filename = os.path.basename(file_path)
        try:
            year = int(re.search(r'\d{4}', filename).group())
        except:
            continue

        print(f"\n📅 Năm {year}:", end=" ")
        all_cols = list(set([col for cols in BLOCK_MAP.values() for col in cols]))

        try:
            chunks = pd.read_csv(file_path, usecols=all_cols, chunksize=200000)
        except ValueError:
            print("⚠️ Bỏ qua")
            continue

        block_data = {k: [] for k in BLOCK_MAP.keys()}
        total_students = 0

        for chunk in chunks:
            total_students += len(chunk)
            for block, cols in BLOCK_MAP.items():
                if not all(col in chunk.columns for col in cols):
                    continue
                temp = chunk.dropna(subset=cols)
                if not temp.empty:
                    scores = temp[cols].sum(axis=1).tolist()
                    block_data[block].extend(scores)

        blocks_processed = 0
        for block, scores in block_data.items():
            if not scores:
                continue
            scores_np = np.array(scores)
            scores_np.sort()
            scores_np = scores_np[::-1]

            df_score = pd.DataFrame({'score': scores_np})
            df_score['rank'] = df_score['score'].rank(method='min', ascending=False)
            total = len(df_score)

            lookup = df_score.groupby('score')['rank'].min().reset_index()
            lookup['percentile'] = (lookup['rank'] / total) * 100
            lookup_dict[(year, block)] = lookup.sort_values('score')
            blocks_processed += 1

        print(f"{total_students:,} thí sinh, {blocks_processed} tổ hợp ✅")
        year_stats.append({'Năm': year, 'Thí sinh': total_students, 'Tổ hợp': blocks_processed})

    print(f"\n✅ Đã xử lý {len(lookup_dict)} cặp (Năm, Tổ hợp)")
    return lookup_dict, pd.DataFrame(year_stats)

# Run
lookup_map, stats_df = build_percentile_lookup('diem_thi_thptqg')

In [ ]:
# Visualize thống kê điểm thi
if not stats_df.empty:
    fig = px.bar(stats_df, x='Năm', y='Thí sinh',
                 title='👥 Số lượng Thí sinh THPT theo Năm',
                 text='Thí sinh',
                 color='Thí sinh',
                 color_continuous_scale='Blues')
    fig.update_traces(texttemplate='%{text:,.0f}', textposition='outside')
    fig.update_layout(height=400)
    fig.show()

In [ ]:
# Visualize phân phối điểm 2025
if (2025, 'B00') in lookup_map:
    sample = lookup_map[(2025, 'B00')]

    fig = make_subplots(rows=1, cols=2,
                        subplot_titles=['Phân phối điểm B00 (2025)', 'Điểm → Percentile'])

    fig.add_trace(go.Histogram(x=sample['score'], nbinsx=50, name='Số lượng',
                               marker_color='#636EFA'), row=1, col=1)
    fig.add_trace(go.Scatter(x=sample['score'], y=sample['percentile'],
                             mode='lines', name='Percentile', line=dict(color='#EF553B')),
                  row=1, col=2)

    fig.update_layout(height=400, title_text='📊 Phân tích Phổ điểm B00 - 2025')
    fig.show()

    print("\n📋 Bảng tra cứu mẫu:")
    for score in [27, 25, 23, 20]:
        pct = sample[sample['score'] >= score].iloc[0]['percentile']
        print(f"  Điểm {score} = Top {pct:.1f}%")

## 🔄 6. Normalize Benchmark Data

In [ ]:
def expand_multiple_blocks(df):
    """Tách dòng có nhiều tổ hợp"""
    expanded_rows = []
    for _, row in df.iterrows():
        to_hop = str(row['to_hop_mon']).strip()
        blocks = [b.strip() for b in to_hop.split(',')]
        for block in blocks:
            if block and block in BLOCK_MAP:
                new_row = row.copy()
                new_row['to_hop_mon'] = block
                expanded_rows.append(new_row)
    return pd.DataFrame(expanded_rows)

def normalize_benchmark(df, lookup_dict):
    """Quy đổi điểm chuẩn sang percentile"""
    print("\n" + "="*60)
    print("📊 QUY ĐỔI ĐIỂM CHUẨN SANG PERCENTILE")
    print("="*60)

    # Tách tổ hợp
    df_expanded = expand_multiple_blocks(df)
    print(f"Sau khi tách tổ hợp: {len(df_expanded):,} dòng")

    # Lọc ngành có 2025
    nganh_2025 = df_expanded[df_expanded['nam'] == 2025][['university_id', 'ma_nganh', 'to_hop_mon']].drop_duplicates()
    print(f"Ngành có data 2025: {len(nganh_2025)}")

    df_filtered = df_expanded.merge(nganh_2025, on=['university_id', 'ma_nganh', 'to_hop_mon'], how='inner')
    print(f"Sau khi lọc: {len(df_filtered):,} dòng")

    # Quy đổi
    def get_percentile(row):
        lookup = lookup_dict.get((row['nam'], str(row['to_hop_mon']).strip()))
        if lookup is None:
            return np.nan
        idx = np.searchsorted(lookup['score'], row['diem_chuan'], side='left')
        return lookup.iloc[idx]['percentile'] if idx < len(lookup) else 0.01

    df_filtered['percentile_rank'] = df_filtered.apply(get_percentile, axis=1)
    df_clean = df_filtered.dropna(subset=['percentile_rank'])

    print(f"\n✅ Dữ liệu training: {len(df_clean):,} dòng")
    return df_clean

# Run
df_train = normalize_benchmark(df_benchmark, lookup_map)

In [ ]:
# Visualize Điểm vs Percentile
sample_data = df_train.sample(min(3000, len(df_train)))

fig = px.scatter(sample_data, x='diem_chuan', y='percentile_rank',
                 color='nam', opacity=0.6,
                 title='📊 Mối quan hệ Điểm chuẩn vs Percentile',
                 labels={'diem_chuan': 'Điểm chuẩn', 'percentile_rank': 'Top %', 'nam': 'Năm'})

fig.update_layout(height=500)
fig.show()

## 🤖 7. Advanced Models Definition

In [ ]:
# Feature Engineering
def calculate_trend(values):
    """Xu hướng (slope)"""
    if len(values) < 2:
        return 0
    x = np.arange(len(values)).reshape(-1, 1)
    model = LinearRegression()
    model.fit(x, values)
    return model.coef_[0]

def calculate_volatility(values):
    """Độ biến động"""
    return np.std(values) if len(values) > 1 else 0

def calculate_momentum(values):
    """Động lượng (thay đổi gần đây)"""
    return values[-1] - values[-2] if len(values) >= 2 else 0

# Prediction Models
def predict_weighted_average(values, alpha=2.0):
    """Weighted Average với exponential weights"""
    n = len(values)
    if n == 0:
        return None, None
    weights = np.exp(np.linspace(0, alpha, n))
    pred = np.sum(values * weights) / weights.sum()
    weighted_var = np.sum(weights * (values - pred)**2) / weights.sum()
    ci = 1.96 * np.sqrt(weighted_var) if weighted_var > 0 else 0
    return pred, ci

def predict_ets(values):
    """Exponential Smoothing"""
    try:
        if len(values) < 4:
            return None, None
        model = ExponentialSmoothing(values, trend='add', seasonal=None)
        fit = model.fit()
        pred = fit.forecast(1)[0]
        residuals = values - fit.fittedvalues
        ci = 1.96 * np.std(residuals)
        return pred, ci
    except:
        return None, None

def predict_arima(values):
    """ARIMA(1,1,1)"""
    try:
        if len(values) < 5:
            return None, None
        model = ARIMA(values, order=(1, 1, 1))
        fit = model.fit()
        forecast = fit.get_forecast(steps=1)
        pred = forecast.predicted_mean[0]
        ci = forecast.conf_int().iloc[0, 1] - pred
        return pred, ci
    except:
        return None, None

def predict_linear_regression(values):
    """Linear Regression"""
    try:
        if len(values) < 3:
            return None, None
        x = np.arange(len(values)).reshape(-1, 1)
        model = LinearRegression()
        model.fit(x, values)
        pred = model.predict([[len(values)]])[0]
        residuals = values - model.predict(x)
        ci = 1.96 * np.std(residuals)
        return pred, ci
    except:
        return None, None

print("✅ Đã định nghĩa 4 models: WA, ETS, ARIMA, LR")

In [ ]:
def ensemble_predict(values):
    """Ensemble prediction - Kết hợp nhiều models"""
    predictions = {}
    confidence_intervals = {}

    # Run all models
    for name, func in [('WA', predict_weighted_average), ('ETS', predict_ets),
                       ('ARIMA', predict_arima), ('LR', predict_linear_regression)]:
        pred, ci = func(values)
        if pred is not None:
            predictions[name] = pred
            confidence_intervals[name] = ci if ci and ci > 0 else 0.1

    if not predictions:
        return np.mean(values), np.std(values) * 1.96, 'MEAN', {}

    # Weighted ensemble (weight = 1/CI)
    total_weight = 0
    weighted_pred = 0

    for model, pred in predictions.items():
        weight = 1 / confidence_intervals[model]
        weighted_pred += pred * weight
        total_weight += weight

    ensemble_pred = weighted_pred / total_weight if total_weight > 0 else np.mean(list(predictions.values()))
    best_model = min(confidence_intervals.keys(), key=lambda k: confidence_intervals[k])
    ensemble_ci = np.mean(list(confidence_intervals.values()))

    return ensemble_pred, ensemble_ci, best_model, predictions

print("✅ Đã định nghĩa Ensemble Model")

## 🏋️ 8. Training với Cross-Validation

In [ ]:
def time_series_cv(values, n_splits=3):
    """Time Series Cross-Validation"""
    if len(values) < n_splits + 2:
        return None

    errors = defaultdict(list)

    for i in range(n_splits):
        split_point = len(values) - n_splits + i
        if split_point < 3:
            continue

        train = values[:split_point]
        test = values[split_point]

        for name, func in [('WA', predict_weighted_average), ('ETS', predict_ets),
                           ('ARIMA', predict_arima), ('LR', predict_linear_regression)]:
            pred, _ = func(train)
            if pred is not None:
                errors[name].append(abs(pred - test))

    return {k: np.mean(v) if v else float('inf') for k, v in errors.items()}

print("✅ Đã định nghĩa Time Series Cross-Validation")

In [ ]:
def train_advanced(df_labeled):
    """Advanced Training với Ensemble & CV"""
    print("\n" + "="*60)
    print("🤖 ADVANCED MODEL TRAINING")
    print("="*60)

    groups = df_labeled.groupby(['university_id', 'ma_nganh', 'to_hop_mon'])

    forecast_results = {}
    confidence_results = {}
    model_selection = {}
    feature_store = {}

    stats = defaultdict(int)
    all_errors = defaultdict(list)

    total_groups = len(groups)
    print(f"\n📊 Training {total_groups} nhóm ngành...\n")

    for i, (name, group) in enumerate(groups):
        if (i + 1) % 50 == 0:
            print(f"  ⏳ {i+1}/{total_groups} ({100*(i+1)/total_groups:.0f}%)", end="\r")

        group = group.sort_values('nam')
        percentiles = group['percentile_rank'].values

        # Extract features
        features = {
            'mean': np.mean(percentiles),
            'std': np.std(percentiles) if len(percentiles) > 1 else 0,
            'trend': calculate_trend(percentiles),
            'volatility': calculate_volatility(percentiles),
            'momentum': calculate_momentum(percentiles),
            'n_years': len(percentiles)
        }
        feature_store[name] = features

        # Cross-validation
        cv_errors = time_series_cv(percentiles)

        if cv_errors and len(percentiles) >= 4:
            for model, err in cv_errors.items():
                all_errors[model].append(err)

            # Ensemble prediction
            pred, ci, best_model, _ = ensemble_predict(percentiles)
        else:
            pred, ci = predict_weighted_average(percentiles)
            if pred is None:
                pred = np.mean(percentiles)
                ci = np.std(percentiles) * 1.96
            best_model = 'WA'
            stats['WA_only'] += 1

        # Safety factor
        adjusted_pred = pred * 0.95

        forecast_results[name] = adjusted_pred
        confidence_results[name] = ci if ci else 0
        model_selection[name] = best_model
        stats[best_model] += 1

    print(f"\n\n✅ Đã train {len(forecast_results)} nhóm!")
    print(f"\n📊 Model Selection:")
    for model, count in sorted(stats.items()):
        print(f"   {model}: {count} ({100*count/total_groups:.1f}%)")

    if all_errors:
        print(f"\n📉 CV Error (MAE):")
        for model, errors in all_errors.items():
            if errors:
                print(f"   {model}: {np.mean(errors):.3f} ± {np.std(errors):.3f}")

    analytics = {
        'predictions': forecast_results,
        'confidence_intervals': confidence_results,
        'model_selection': model_selection,
        'features': feature_store,
        'statistics': dict(stats),
        'cv_errors': {k: float(np.mean(v)) for k, v in all_errors.items() if v}
    }

    return forecast_results, confidence_results, analytics

# RUN TRAINING
model_2026, confidence_2026, analytics = train_advanced(df_train)

## 📊 9. Model Evaluation & Visualization

In [ ]:
# 9.1 Model Selection Pie Chart
model_counts = analytics['statistics']
models = [k for k in model_counts.keys() if not k.endswith('_only')]
counts = [model_counts[k] for k in models]

fig = go.Figure(data=[go.Pie(
    labels=models,
    values=counts,
    hole=0.4,
    marker_colors=['#636EFA', '#EF553B', '#00CC96', '#AB63FA', '#FFA15A']
)])

fig.update_layout(
    title='🏆 Model Selection Distribution',
    height=400
)
fig.show()

In [ ]:
# 9.2 CV Error Comparison
cv_errors = analytics.get('cv_errors', {})

if cv_errors:
    fig = go.Figure(go.Bar(
        x=list(cv_errors.values()),
        y=list(cv_errors.keys()),
        orientation='h',
        marker=dict(color=['#636EFA', '#EF553B', '#00CC96', '#AB63FA'][:len(cv_errors)])
    ))

    fig.update_layout(
        title='📉 Cross-Validation Error (MAE)',
        xaxis_title='Mean Absolute Error',
        height=400
    )
    fig.show()

In [ ]:
# 9.3 Confidence Interval Distribution
ci_values = list(confidence_2026.values())

fig = go.Figure()
fig.add_trace(go.Histogram(x=ci_values, nbinsx=40, marker_color='#636EFA', opacity=0.7))
fig.add_vline(x=np.mean(ci_values), line_dash="dash", line_color="red",
              annotation_text=f"Mean: {np.mean(ci_values):.2f}")

fig.update_layout(
    title='📊 Confidence Interval Distribution',
    xaxis_title='Confidence Interval (percentile points)',
    yaxis_title='Frequency',
    height=400
)
fig.show()

In [ ]:
# 9.4 Prediction Distribution
pred_values = list(model_2026.values())

fig = go.Figure()
fig.add_trace(go.Histogram(x=pred_values, nbinsx=40, marker_color='#00CC96', opacity=0.7))
fig.add_vline(x=np.mean(pred_values), line_dash="dash", line_color="red",
              annotation_text=f"Mean: {np.mean(pred_values):.2f}")

fig.update_layout(
    title='🎯 Predicted Percentile Distribution (2026)',
    xaxis_title='Predicted Percentile (Top %)',
    yaxis_title='Frequency',
    height=400
)
fig.show()

In [ ]:
# 9.5 Feature Importance - Trend vs Volatility
features_df = pd.DataFrame([
    {'key': k, 'trend': v['trend'], 'volatility': v['volatility'], 'n_years': v['n_years']}
    for k, v in analytics['features'].items()
])

fig = px.scatter(features_df.sample(min(500, len(features_df))),
                 x='trend', y='volatility', size='n_years',
                 title='📈 Feature Analysis: Trend vs Volatility',
                 labels={'trend': 'Trend (Xu hướng)', 'volatility': 'Volatility (Biến động)'},
                 color='n_years', color_continuous_scale='Viridis')

fig.update_layout(height=500)
fig.show()

## 💾 10. Save & Download Model

In [ ]:
!mkdir -p model_artifacts

# Save
joblib.dump(model_2026, 'model_artifacts/university_ranking_model_2026.pkl')

lookup_2025 = {k: v for k, v in lookup_map.items() if k[0] == 2025}
joblib.dump(lookup_2025, 'model_artifacts/score_distribution_2025.pkl')

joblib.dump(analytics, 'model_artifacts/model_analytics.pkl')

# Save confidence intervals
joblib.dump(confidence_2026, 'model_artifacts/confidence_intervals.pkl')

print("✅ Đã lưu tất cả files!")
print(f"   - Model: {len(model_2026)} ngành")
print(f"   - Lookup: {len(lookup_2025)} tổ hợp")
print(f"   - Analytics: Full statistics")

In [ ]:
# Lưu các files cần thiết
print("💾 Đã lưu tất cả models vào thư mục model_artifacts/")

## 🎯 11. Demo: Tìm Ngành Phù Hợp

In [ ]:
def get_user_percentile(score, block, lookup_dict):
    """Quy đổi điểm user sang percentile"""
    key = (2025, block)
    if key not in lookup_dict:
        return None
    table = lookup_dict[key]
    idx = np.searchsorted(table['score'], score, side='left')
    return table.iloc[idx]['percentile'] if idx < len(table) else 0.01

def find_suitable_majors(student_scores, model, confidence, lookup, df_info, priority=0, top_n=30):
    """Tìm ngành phù hợp với confidence interval"""
    results = []

    for key, predicted_percentile in model.items():
        university_id, ma_nganh, block = key

        if block not in BLOCK_MAP:
            continue

        subjects = BLOCK_MAP[block]
        if not all(s in student_scores for s in subjects):
            continue

        diem_cua_ban = sum(student_scores[s] for s in subjects) + priority
        student_percentile = get_user_percentile(diem_cua_ban, block, lookup)

        if student_percentile is None:
            continue

        khoang_cach = predicted_percentile - student_percentile
        ci = confidence.get(key, 0)

        # Lấy info
        info = df_info[(df_info['university_id'] == university_id) & (df_info['ma_nganh'] == ma_nganh)]
        if info.empty:
            continue
        info = info.iloc[0]

        # Đánh giá
        if khoang_cach > 10:
            kha_nang = "🟢 Rất cao"
            score = 5
        elif khoang_cach > 2:
            kha_nang = "🟢 Cao"
            score = 4
        elif khoang_cach > 0:
            kha_nang = "🟡 Trung bình"
            score = 3
        elif khoang_cach > -2:
            kha_nang = "🟠 Thấp"
            score = 2
        else:
            kha_nang = "🔴 Rất thấp"
            score = 1

        results.append({
            'Trường': info['ten_truong'],
            'Ngành': info['ten_nganh'],
            'Tổ hợp': block,
            'Điểm bạn': diem_cua_ban,
            'Khả năng': kha_nang,
            'CI ±': round(ci, 1),
            'score': score
        })

    df_result = pd.DataFrame(results).sort_values('score', ascending=False)
    return df_result.head(top_n)

print("✅ Đã định nghĩa hàm tìm ngành!")

In [ ]:
# === NHẬP ĐIỂM CỦA BẠN ===
print("="*50)
print("🎓 DEMO: TÌM NGÀNH PHÙ HỢP")
print("="*50)

student_scores = {
    'toan': 8.5,
    'vat_ly': 8.0,
    'hoa_hoc': 8.5,
    'sinh_hoc': 7.5,
    'ngu_van': 7.0,
    'ngoai_ngu': 8.0
}
priority = 0

print("\n📝 Điểm của bạn:")
for s, score in student_scores.items():
    print(f"   {s}: {score}")

print("\n📊 Điểm tổ hợp:")
for block, subjects in BLOCK_MAP.items():
    total = sum(student_scores.get(s, 0) for s in subjects) + priority
    print(f"   {block}: {total:.1f}")

In [ ]:
# Tìm ngành
suitable = find_suitable_majors(student_scores, model_2026, confidence_2026, lookup_2025, df_benchmark, priority)

if not suitable.empty:
    print("\n🎯 NGÀNH PHÙ HỢP:")
    print("="*80)

    display(suitable[['Trường', 'Ngành', 'Tổ hợp', 'Điểm bạn', 'Khả năng', 'CI ±']])

    # Pie chart
    chance_counts = suitable['Khả năng'].value_counts()
    fig = px.pie(values=chance_counts.values, names=chance_counts.index,
                 title='📊 Phân bố Cơ hội đậu', hole=0.4)
    fig.show()
else:
    print("❌ Không tìm thấy ngành phù hợp")

## 📊 KẾT LUẬN PHẦN 1: Percentile Prediction

Phần trên đã hoàn thành **Mục tiêu 1**: Dự đoán percentile điểm chuẩn 2026

---

# 🎯 PHẦN 2: ADMISSION PROBABILITY MODEL (v3 Approach)

## Tại sao cần phần này?

**Phần 1** cho biết: "Ngành Y khoa sẽ lấy Top 0.5%"  
**Phần 2** cho biết: "Với điểm 27, bạn có 65% cơ hội đậu Y khoa"

### 💡 Key Insight - Tại sao v3 khác v1/v2?

```
❌ v1/v2 (SAI):
   Features: [điểm_thi, gap, percentile, ...]
   gap = điểm_thi - điểm_chuẩn_năm_đó
   
   Vấn đề: Khi predict 2026, ta KHÔNG CÓ điểm_chuẩn_2026!

✅ v3 (ĐÚNG):
   Features: [student_percentile, percentile_required, university, major, ...]
   
   Cách hoạt động: Model học pattern từ lịch sử
   "Ngành Y khoa thường yêu cầu Top 0.5%"
   "Thí sinh Top 0.3% có 80% cơ hội đậu ngành yêu cầu Top 0.5%"
```

In [ ]:
# =====================================================
# BƯỚC 1: CHUẨN BỊ DỮ LIỆU CHO ADMISSION PROBABILITY
# =====================================================

print("="*60)
print("🎯 ADMISSION PROBABILITY MODEL v3 - REALISTIC APPROACH")
print("="*60)

# Load lại dữ liệu điểm thi (nếu chưa có)
all_score_data = {}  # {(year, block): scores array}

print("\n[STEP 1] Loading điểm thi THPT (2020-2025)...")
for f in sorted(glob.glob(os.path.join(SCORE_FOLDER, '*.csv'))):
    year = int(re.search(r'\d{4}', os.path.basename(f)).group())
    if year < 2020:
        continue

    print(f"   -> Năm {year}...", end=" ")
    df_scores = pd.read_csv(f)

    for block, cols in BLOCK_MAP.items():
        if all(c in df_scores.columns for c in cols):
            temp = df_scores.dropna(subset=cols)
            scores = temp[cols].sum(axis=1).values
            all_score_data[(year, block)] = scores
    print(f"✅ ({len(df_scores):,} thí sinh)")

print(f"\n✅ Loaded {len(all_score_data)} cặp (năm, tổ hợp)")

In [ ]:
# =====================================================
# BƯỚC 2: TÍNH PERCENTILE CHO ĐIỂM CHUẨN
# =====================================================
# Key insight: Mỗi ngành "yêu cầu" một percentile nhất định
# Ví dụ: Y khoa cần Top 0.5%, Dược cần Top 2%

print("\n[STEP 2] Tính percentile_required cho mỗi ngành...")

def get_percentile_for_score(score, year, block):
    """Tính percentile của điểm trong phân phối năm đó"""
    key = (year, block)
    if key not in all_score_data:
        return None
    scores = all_score_data[key]
    # Percentile = % thí sinh có điểm <= score
    return (scores <= score).sum() / len(scores) * 100

# Load điểm chuẩn
df_benchmark_v3 = pd.read_csv(BENCHMARK_FILE)
df_benchmark_v3 = df_benchmark_v3[df_benchmark_v3['nam'] >= 2020]

# Thêm cột percentile_required
df_benchmark_v3['percentile_required'] = df_benchmark_v3.apply(
    lambda row: get_percentile_for_score(row['diem_chuan'], row['nam'], row['to_hop_mon']),
    axis=1
)
df_benchmark_v3 = df_benchmark_v3.dropna(subset=['percentile_required'])

print(f"   -> {len(df_benchmark_v3)} ngành có percentile_required")

# Hiển thị ví dụ
print("\n📋 Ví dụ percentile_required:")
sample = df_benchmark_v3[['ten_truong', 'ten_nganh', 'to_hop_mon', 'diem_chuan', 'percentile_required', 'nam']].head(5)
print(sample.to_string(index=False))

# =====================================================
# BƯỚC 3: TẠO TRAINING DATA VỚI STRATIFIED SAMPLING
# =====================================================
# Cách tiếp cận v3 + Stratified Sampling:
# - Sample điểm thi thật từ phân phối theo ZONES
# - Tính student_percentile (vị trí của thí sinh)
# - Label: 1 nếu student_percentile >= percentile_required, 0 nếu không

print("\n[STEP 3] Tạo training data với Stratified Sampling...")

def stratified_sample(scores, diem_chuan, total_samples=300):
    """
    Stratified Sampling: Lấy mẫu thông minh theo 4 zones
    - Zone 1 (rớt chắc): gap < -5     -> 15% samples
    - Zone 2 (có thể rớt): -5 <= gap < -1  -> 25% samples
    - Zone 3 (ranh giới): -1 <= gap < +3   -> 40% samples
    - Zone 4 (đậu chắc): gap >= +3    -> 20% samples

    Ưu điểm:
    - Giữ lại extreme cases (model vẫn học điểm cao=đậu, thấp=rớt)
    - Tập trung vào vùng ranh giới (nơi model cần học phân biệt)
    - Model generalize tốt hơn khi deploy
    """
    gaps = scores - diem_chuan

    # Phân chia zones
    zone1_mask = gaps < -5
    zone2_mask = (gaps >= -5) & (gaps < -1)
    zone3_mask = (gaps >= -1) & (gaps < 3)
    zone4_mask = gaps >= 3

    zone_scores = [
        scores[zone1_mask],
        scores[zone2_mask],
        scores[zone3_mask],
        scores[zone4_mask]
    ]

    # Tỷ lệ samples cho mỗi zone
    zone_ratios = [0.15, 0.25, 0.40, 0.20]

    sampled_all = []
    for zone_data, ratio in zip(zone_scores, zone_ratios):
        n_want = int(total_samples * ratio)
        if len(zone_data) > 0:
            n_take = min(n_want, len(zone_data))
            sampled = np.random.choice(zone_data, size=n_take, replace=False)
            sampled_all.extend(sampled)

    return np.array(sampled_all)

training_data = []
sample_per_major = 300  # Số samples mỗi ngành

for _, row in df_benchmark_v3.iterrows():
    block = str(row['to_hop_mon']).strip()
    nam = row['nam']
    key = (nam, block)

    if key not in all_score_data:
        continue

    scores = all_score_data[key]
    diem_chuan = row['diem_chuan']
    percentile_required = row['percentile_required']
    university_id = row['university_id']
    ma_nganh = row['ma_nganh']

    # STRATIFIED SAMPLING thay vì random sampling
    sampled = stratified_sample(scores, diem_chuan, sample_per_major)

    for score in sampled:
        # Tính student_percentile
        student_percentile = (scores <= score).sum() / len(scores) * 100

        # Label: đậu nếu percentile >= percentile_required
        # (percentile cao = top đầu, percentile_required thấp = ngành khó)
        label = 1 if student_percentile >= percentile_required else 0

        training_data.append({
            'university_id': university_id,
            'ma_nganh': ma_nganh,
            'to_hop_mon': block,
            'student_percentile': student_percentile,
            'percentile_required': percentile_required,
            'label': label
        })

df_train_prob = pd.DataFrame(training_data)
print(f"\n✅ Tạo được {len(df_train_prob):,} training samples")
print(f"   -> Đậu: {(df_train_prob['label']==1).sum():,} ({(df_train_prob['label']==1).mean()*100:.1f}%)")
print(f"   -> Trượt: {(df_train_prob['label']==0).sum():,} ({(df_train_prob['label']==0).mean()*100:.1f}%)")

In [ ]:
# =====================================================
# BƯỚC 3: TẠO TRAINING DATA VỚI STRATIFIED SAMPLING
# =====================================================
# Cách tiếp cận v3 + Stratified Sampling:
# - Sample điểm thi thật từ phân phối theo ZONES
# - Tính student_percentile (vị trí của thí sinh)
# - Label: 1 nếu student_percentile >= percentile_required, 0 nếu không

print("\n[STEP 3] Tạo training data với Stratified Sampling...")

def stratified_sample(scores, diem_chuan, total_samples=300):
    """
    Stratified Sampling: Lấy mẫu thông minh theo 4 zones
    - Zone 1 (rớt chắc): gap < -5     -> 15% samples
    - Zone 2 (có thể rớt): -5 <= gap < -1  -> 25% samples
    - Zone 3 (ranh giới): -1 <= gap < +3   -> 40% samples
    - Zone 4 (đậu chắc): gap >= +3    -> 20% samples

    Ưu điểm:
    - Giữ lại extreme cases (model vẫn học điểm cao=đậu, thấp=rớt)
    - Tập trung vào vùng ranh giới (nơi model cần học phân biệt)
    - Model generalize tốt hơn khi deploy
    """
    gaps = scores - diem_chuan

    # Phân chia zones
    zone1_mask = gaps < -5
    zone2_mask = (gaps >= -5) & (gaps < -1)
    zone3_mask = (gaps >= -1) & (gaps < 3)
    zone4_mask = gaps >= 3

    zone_scores = [
        scores[zone1_mask],
        scores[zone2_mask],
        scores[zone3_mask],
        scores[zone4_mask]
    ]

    # Tỷ lệ samples cho mỗi zone
    zone_ratios = [0.15, 0.25, 0.40, 0.20]

    sampled_all = []
    for zone_data, ratio in zip(zone_scores, zone_ratios):
        n_want = int(total_samples * ratio)
        if len(zone_data) > 0:
            n_take = min(n_want, len(zone_data))
            sampled = np.random.choice(zone_data, size=n_take, replace=False)
            sampled_all.extend(sampled)

    return np.array(sampled_all)

training_data = []
sample_per_major = 300  # Số samples mỗi ngành

for _, row in df_benchmark_v3.iterrows():
    block = str(row['to_hop_mon']).strip()
    nam = row['nam']
    key = (nam, block)

    if key not in all_score_data:
        continue

    scores = all_score_data[key]
    diem_chuan = row['diem_chuan']
    percentile_required = row['percentile_required']
    university_id = row['university_id']
    ma_nganh = row['ma_nganh']

    # STRATIFIED SAMPLING thay vì random sampling
    sampled = stratified_sample(scores, diem_chuan, sample_per_major)

    for score in sampled:
        # Tính student_percentile
        student_percentile = (scores <= score).sum() / len(scores) * 100

        # Label: đậu nếu percentile >= percentile_required
        # (percentile cao = top đầu, percentile_required thấp = ngành khó)
        label = 1 if student_percentile >= percentile_required else 0

        training_data.append({
            'university_id': university_id,
            'ma_nganh': ma_nganh,
            'to_hop_mon': block,
            'student_percentile': student_percentile,
            'percentile_required': percentile_required,
            'label': label
        })

df_train_prob = pd.DataFrame(training_data)
print(f"\n✅ Tạo được {len(df_train_prob):,} training samples")
print(f"   -> Đậu: {(df_train_prob['label']==1).sum():,} ({(df_train_prob['label']==1).mean()*100:.1f}%)")
print(f"   -> Trượt: {(df_train_prob['label']==0).sum():,} ({(df_train_prob['label']==0).mean()*100:.1f}%)")

### 💡 Giải thích Logic Training Data

```
Ngành Y khoa Hà Nội yêu cầu Top 0.5% (percentile_required = 99.5)

Thí sinh A: Điểm 27.5 = Top 0.3% (student_percentile = 99.7)
→ 99.7 >= 99.5 → Label = 1 (Đậu ✅)

Thí sinh B: Điểm 25.0 = Top 2% (student_percentile = 98.0)  
→ 98.0 < 99.5 → Label = 0 (Trượt ❌)
```

**Tại sao cách này realistic?**
- Không cần biết điểm chuẩn 2026
- Model học pattern: "Ngành nào thường lấy Top bao nhiêu %"
- Có thể predict: "Với vị trí Top X% của bạn, khả năng đậu ngành Y là Z%"

### 💡 Giải thích Stratified Sampling

```
─────────────────────────────────────────────────────
                    PHÂN BỐ ĐIỂM                       
─────────────┬─────────────┬─────────────┬──────────
   Zone 1    │   Zone 2    │   Zone 3    │  Zone 4  
  Rớt chắc   │  Có thể rớt │  Ranh giới  │ Đậu chắc
   (<-5)     │   (-5→-1)   │   (-1→+3)   │  (>+3)   
─────────────┼─────────────┼─────────────┼──────────
  Sample 15% │  Sample 25% │  Sample 40% │Sample 20%
─────────────┴─────────────┴─────────────┴──────────
```

**Tại sao Stratified Sampling tốt hơn Random Sampling?**

| Gap (điểm - ĐC) | Random Sampling | Stratified Sampling |
|-----------------|-----------------|---------------------|
| +10 (đậu chắc)  | 🟠 Ít samples    | ✅ Giữ 20%           |
| +2 (sát mép)     | 🟠 Random       | ✅ Giữ 40%           |
| -2 (sát mép)     | 🟠 Random       | ✅ Giữ 40%           |
| -10 (rớt chắc)  | 🟠 Nhiều samples | ✅ Giữ 15%           |

**Ưu điểm:**
1. **Giữ lại extreme cases**: Model vẫn học "điểm cao = đậu", "điểm thấp = rớt"
2. **Tập trung vùng quan trọng**: Zone 2 & 3 (vùng ranh giới) được sample nhiều hơn
3. **Robust khi deploy**: Model xử lý được mọi input, không bị "out of distribution"

In [ ]:
# =====================================================
# BƯỚC 4: FEATURE ENGINEERING
# =====================================================

print("\n[STEP 4] Feature Engineering...")

le_university = LabelEncoder()
le_nganh = LabelEncoder()
le_block = LabelEncoder()

df_train_prob['uni_enc'] = le_university.fit_transform(df_train_prob['university_id'].astype(str))
df_train_prob['nganh_enc'] = le_nganh.fit_transform(df_train_prob['ma_nganh'].astype(str))
df_train_prob['block_enc'] = le_block.fit_transform(df_train_prob['to_hop_mon'])

# QUAN TRỌNG: Features KHÔNG bao gồm điểm chuẩn hay gap!
feature_cols = [
    'student_percentile',    # Vị trí của thí sinh (Top bao nhiêu %)
    'percentile_required',   # Ngành yêu cầu Top bao nhiêu % (từ lịch sử)
    'uni_enc',               # Encoded trường
    'nganh_enc',             # Encoded ngành
    'block_enc',             # Encoded tổ hợp
]

X = df_train_prob[feature_cols].values
y = df_train_prob['label'].values

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

print(f"   ✅ Features: {feature_cols}")
print(f"   ✅ X shape: {X_scaled.shape}")

In [ ]:
# =====================================================
# BƯỚC 5: TRAIN CLASSIFICATION MODELS
# =====================================================

print("\n[STEP 5] Training Classification Models...")

X_train, X_test, y_train, y_test = train_test_split(
    X_scaled, y, test_size=0.2, random_state=42, stratify=y
)
print(f"   Train: {len(X_train):,}, Test: {len(X_test):,}")

# Train Gradient Boosting (tốt cho bài toán này)
print("\n   Training Gradient Boosting...")
gb_model = GradientBoostingClassifier(
    n_estimators=100,
    max_depth=5,
    learning_rate=0.1,
    random_state=42
)
gb_model.fit(X_train, y_train)

# Evaluate
y_pred = gb_model.predict(X_test)
y_prob = gb_model.predict_proba(X_test)[:, 1]

accuracy = accuracy_score(y_test, y_pred)
roc_auc = roc_auc_score(y_test, y_prob)

print(f"\n📊 RESULTS:")
print(f"   Accuracy: {accuracy:.4f}")
print(f"   ROC-AUC: {roc_auc:.4f}")

# Cross validation
cv_scores = cross_val_score(gb_model, X_scaled, y, cv=5, scoring='roc_auc')
print(f"   CV Score: {cv_scores.mean():.4f} (±{cv_scores.std()*2:.4f})")

# Feature importance
print(f"\n📈 Feature Importance:")
for i, col in enumerate(feature_cols):
    print(f"   {col}: {gb_model.feature_importances_[i]:.4f}")

In [ ]:
# Visualize Feature Importance
fig = go.Figure(go.Bar(
    x=gb_model.feature_importances_,
    y=feature_cols,
    orientation='h',
    marker=dict(color=gb_model.feature_importances_, colorscale='Blues')
))

fig.update_layout(
    title='📊 Feature Importance - Admission Probability Model',
    xaxis_title='Importance',
    height=400
)
fig.show()

In [ ]:
# =====================================================
# BƯỚC 6: SỬ DỤNG ENSEMBLE MODEL 2026 (thay vì trung bình lịch sử)
# =====================================================
# Lập luận: Dùng output của Ensemble (WA, ETS, ARIMA, LR) làm percentile_required
# thay vì dùng trung bình lịch sử - vì Ensemble đã predict cho 2026!

print("\n[STEP 6] Sử dụng Ensemble Model 2026...")

# model_2026 đã được train ở phần 1 (Ensemble: WA, ETS, ARIMA, LR)
# Format: {(university_id, ma_nganh, to_hop_mon): predicted_percentile_2026}

if 'model_2026' in dir() and model_2026:
    print("   ✅ Sử dụng Ensemble output (model_2026)")

    # Tạo DataFrame từ model_2026
    ensemble_percentile = pd.DataFrame([
        {'university_id': k[0], 'ma_nganh': k[1], 'to_hop_mon': k[2], 'predicted_percentile_2026': v}
        for k, v in model_2026.items()
    ])
    print(f"   -> {len(ensemble_percentile)} dự đoán percentile 2026 từ Ensemble")

    # Merge với thông tin điểm chuẩn để có avg_diem_chuan
    avg_diem = df_benchmark_v3.groupby(['university_id', 'ma_nganh', 'to_hop_mon']).agg({
        'diem_chuan': 'mean'
    }).reset_index()
    avg_diem.columns = ['university_id', 'ma_nganh', 'to_hop_mon', 'avg_diem_chuan']

    avg_percentile = ensemble_percentile.merge(avg_diem, on=['university_id', 'ma_nganh', 'to_hop_mon'], how='left')
    avg_percentile.columns = ['university_id', 'ma_nganh', 'to_hop_mon', 'avg_percentile_required', 'avg_diem_chuan']
else:
    print("   ⚠️ Không tìm thấy model_2026, sử dụng trung bình lịch sử")
    avg_percentile = df_benchmark_v3.groupby(['university_id', 'ma_nganh', 'to_hop_mon']).agg({
        'percentile_required': 'mean',
        'diem_chuan': 'mean'
    }).reset_index()
    avg_percentile.columns = ['university_id', 'ma_nganh', 'to_hop_mon', 'avg_percentile_required', 'avg_diem_chuan']

print(f"   -> {len(avg_percentile)} ngành")

# Hiển thị mẫu
print("\n📋 Top 10 ngành khó nhất (yêu cầu percentile cao nhất):")
top_hard = avg_percentile.nsmallest(10, 'avg_percentile_required').copy()

# Merge để lấy tên
top_hard = top_hard.merge(
    df_benchmark_v3[['university_id', 'ma_nganh', 'ten_truong', 'ten_nganh']].drop_duplicates(),
    on=['university_id', 'ma_nganh']
)
print(top_hard[['ten_truong', 'ten_nganh', 'to_hop_mon', 'avg_diem_chuan', 'avg_percentile_required']].to_string(index=False))

In [ ]:
# =====================================================
# BƯỚC 7: LƯU TẤT CẢ MODELS
# =====================================================

print("\n[STEP 7] Lưu models...")

# Save classification model
joblib.dump(gb_model, os.path.join(OUTPUT_DIR, 'admission_probability_model.pkl'))
print(f"   ✅ Saved: admission_probability_model.pkl")

# Save encoders
encoders = {
    'university': le_university,
    'nganh': le_nganh,
    'block': le_block,
    'feature_cols': feature_cols
}
joblib.dump(encoders, os.path.join(OUTPUT_DIR, 'admission_encoders.pkl'))
print(f"   ✅ Saved: admission_encoders.pkl")

# Save scaler
joblib.dump(scaler, os.path.join(OUTPUT_DIR, 'admission_scaler.pkl'))
print(f"   ✅ Saved: admission_scaler.pkl")

# Save avg_percentile lookup
joblib.dump(avg_percentile, os.path.join(OUTPUT_DIR, 'major_percentile_lookup.pkl'))
print(f"   ✅ Saved: major_percentile_lookup.pkl")

# Save score data 2025 để dùng cho web app
score_data_2025 = {k: v for k, v in all_score_data.items() if k[0] == 2025}
joblib.dump(score_data_2025, os.path.join(OUTPUT_DIR, 'score_data_2025.pkl'))
print(f"   ✅ Saved: score_data_2025.pkl")

print("\n🎉 Đã lưu tất cả models!")

## 🎯 DEMO: Dự đoán Xác suất Đậu 2026

In [ ]:
def predict_admission_probability(diem, block, university_id, ma_nganh):
    """
    Dự đoán xác suất đậu cho thí sinh

    Args:
        diem: Điểm tổ hợp của thí sinh
        block: Tổ hợp môn (VD: 'B00')
        university_id: ID trường
        ma_nganh: Mã ngành

    Returns:
        prob: Xác suất đậu (0-1)
        student_percentile: Vị trí của thí sinh
        percentile_required: Ngành yêu cầu
    """
    # 1. Tính student_percentile từ điểm 2025
    key = (2025, block)
    if key not in all_score_data:
        return None, None, None

    scores = all_score_data[key]
    student_percentile = (scores <= diem).sum() / len(scores) * 100

    # 2. Lấy percentile_required từ lookup
    lookup_row = avg_percentile[
        (avg_percentile['university_id'] == university_id) &
        (avg_percentile['ma_nganh'] == ma_nganh) &
        (avg_percentile['to_hop_mon'] == block)
    ]

    if lookup_row.empty:
        return None, student_percentile, None

    percentile_required = lookup_row['avg_percentile_required'].values[0]

    # 3. Encode features
    try:
        uni_enc = le_university.transform([str(university_id)])[0] if str(university_id) in le_university.classes_ else 0
        nganh_enc = le_nganh.transform([str(ma_nganh)])[0] if str(ma_nganh) in le_nganh.classes_ else 0
        block_enc = le_block.transform([block])[0] if block in le_block.classes_ else 0
    except:
        uni_enc, nganh_enc, block_enc = 0, 0, 0

    # 4. Predict
    features = np.array([[student_percentile, percentile_required, uni_enc, nganh_enc, block_enc]])
    features_scaled = scaler.transform(features)
    prob = gb_model.predict_proba(features_scaled)[0][1]

    return prob, student_percentile, percentile_required

print("✅ Đã định nghĩa hàm predict_admission_probability()")

In [ ]:
# =====================================================
# DEMO: Thử nghiệm dự đoán
# =====================================================

print("="*60)
print("🎯 DEMO: DỰ ĐOÁN XÁC SUẤT ĐẬU 2026")
print("="*60)

# Lấy danh sách một số ngành để test
sample_majors = df_benchmark_v3[['university_id', 'ma_nganh', 'ten_truong', 'ten_nganh', 'to_hop_mon']].drop_duplicates().head(5)
print("\n📋 Một số ngành để test:")
print(sample_majors.to_string(index=False))

# Test cases
print("\n" + "-"*60)
print("📊 KẾT QUẢ DỰ ĐOÁN:")
print("-"*60)

test_cases = [
    (27.0, 'B00', 215, '7720101', 'Điểm cao (27.0)'),
    (25.0, 'B00', 215, '7720101', 'Điểm trung bình (25.0)'),
    (23.0, 'B00', 215, '7720101', 'Điểm thấp (23.0)'),
]

for diem, block, uni_id, ma_nganh, desc in test_cases:
    prob, student_pct, required_pct = predict_admission_probability(diem, block, uni_id, ma_nganh)

    if prob is not None:
        # Lấy tên ngành
        info = df_benchmark_v3[(df_benchmark_v3['university_id']==uni_id) & (df_benchmark_v3['ma_nganh']==ma_nganh)]
        ten_nganh = info['ten_nganh'].iloc[0] if not info.empty else "Unknown"

        print(f"\n🎓 {ten_nganh} - {desc}")
        print(f"   Điểm: {diem} | Tổ hợp: {block}")
        print(f"   Bạn thuộc Top: {100-student_pct:.2f}%")
        print(f"   Ngành yêu cầu Top: {100-required_pct:.2f}%")
        print(f"   >>> XÁC SUẤT ĐẬU: {prob*100:.1f}%")
    else:
        print(f"\n❌ {desc}: Không thể dự đoán")

---
## 🎉 HOÀN THÀNH!

### 📊 Tóm tắt 2 Phần:

| Phần | Mục tiêu | Models | Output |
|------|----------|--------|--------|
| **Phần 1** | Dự đoán Percentile 2026 | WA, ETS, ARIMA, LR (Ensemble) | "Y khoa sẽ lấy Top 0.5%" |
| **Phần 2** | Dự đoán Xác suất Đậu | Gradient Boosting Classifier | "Với 27 điểm, bạn có 65% đậu" |

### 💡 Key Takeaway - Tại sao v3 đúng?

**v1/v2 sai** vì dùng `gap = điểm - điểm_chuẩn`:
- Khi train: biết điểm chuẩn → tính được gap
- Khi predict 2026: KHÔNG biết điểm chuẩn → KHÔNG tính được gap!

**v3 đúng** vì chỉ dùng `percentile`:
- Train: học pattern "ngành X thường lấy Top Y%"  
- Predict: chỉ cần biết "bạn đang Top bao nhiêu %" → so sánh với pattern → ra xác suất

### 📁 Files Output:
- `university_ranking_model_2026.pkl` - Percentile predictions
- `admission_probability_model.pkl` - Probability classifier
- `admission_encoders.pkl` - Label encoders
- `admission_scaler.pkl` - StandardScaler
- `major_percentile_lookup.pkl` - Lookup table
- `score_distribution_2025.pkl` - Phân phối điểm
- `model_analytics.pkl` - Analytics đầy đủ

---
**© 2025 Advanced University Admission Predictor V2.0**